<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 从零开始构建字节对编码（BPE）分词器

- 这是一个独立的notebook，旨在从零开始实现字节对编码（BPE）分词算法，该算法广泛应用于 GPT-2 至 GPT-4、Llama 3 等大模型。
- 分词的具体目的请参见[第二章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)；这里的代码为额外材料，用于解释 BPE 算法。
- OpenAI 为训练原始 GPT 模型所实现的 BPE 分词器可以在[此处](https://github.com/openai/gpt-2/blob/master/src/encoder.py)。
- BPE 算法最早由 Philip Gage 于 1994 年提出，详见：“[一种新的数据压缩算法](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)”。
- 目前，大多数项目，包括 Llama 3，使用 OpenAI 开源的 [tiktoken 库](https://github.com/openai/tiktoken)，因为其卓越的计算性能；它支持加载预训练的 GPT-2 和 GPT-4 分词器（Llama 3 模型同样是使用 GPT-4 分词器训练的）。
- 上述案例与我在此notebook中提供有所区别，其一，我们的分词器主要是用于教学目的；其二，我们构建了一个用于训练分词器的函数。
- 还有一个[minBPE](https://github.com/karpathy/minbpe)，可能有更好的性能；与 `minbpe` 不同，书中的代码还支持加载 OpenAI 原始分词器的词汇表与自己的单词表合并。


&nbsp;
# 1. 字节对编码（BPE）的主要思想

- BPE 的主要思想是将文本转换为整数表示（token ID）用于大规模语言模型（LLM）的训练（详见[第二章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)）。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/bpe-overview.webp" width="600px">


&nbsp;
## 1.1 Bits 和 bytes

- 在介绍 BPE 算法之前，我们先来了解一下字节的概念。
- 我们将文本转换为字节数组（毕竟 BPE 代表的是“字节”对编码）：

In [1]:
text = "This is some text"
byte_ary = bytearray(text, "utf-8")
print(byte_ary)

bytearray(b'This is some text')


- 当我们对 `bytearray` 对象调用 `list()` 时，每个字节会被视为一个独立的元素，结果是一个包含字节值对应整数的列表：

In [2]:
ids = list(byte_ary)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


- 这种方式是将文本转换为 LLM 嵌入层中所需的token ID的一种有效方法。
- 然而，这种方法的缺点是，它为每个字符分配一个 ID（对于短文本来说，这将产生大量的 ID！）。
- 也就是说，对于一个包含 17 个字符的输入文本，我们需要使用 17 个token ID 作为 LLM 的输入：

In [3]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


- 如果你之前使用过 LLM，你可能知道 BPE 分词器有一个词汇表，其中我们按照整个词或子词分配ID，按照单词。
- 例如，GPT-2 分词器将相同的文本（"This is some text"）分词为 4 个令牌，而不是 17 个：`1212, 318, 617, 2420`
- 你可以使用互动式的 [tiktoken 应用](https://tiktokenizer.vercel.app/?model=gpt2) 或 [tiktoken 库](https://github.com/openai/tiktoken)进行验证：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="600px">


```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints [1212, 318, 617, 2420]
```

- 由于一个字节由 8 位组成，因此单个字节可以表示 2<sup>8</sup> = 256 个可能的值，范围从 0 到 255。
- 你可以通过执行代码 `bytearray(range(0, 257))` 来确认这一点，代码会提醒你 `ValueError: byte must be in range(0, 256)`。
- 一个 BPE 分词器通常将这 256 个值作为其前 256 个单字符令牌；你可以通过运行以下代码来直观地查看这一点：


```python
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")
"""
prints:
0: !
1: "
2: #
...
255: �  # <---- single character tokens up to here
256:  t
257:  a
...
298: ent
299:  n
"""
```

- 上述代码中，注意到条目 256 和 257 不是单字符值，而是双字符值（一个空格 + 一个字母），这是原始 GPT-2 BPE 分词器的一个小缺点（在 GPT-4 分词器中已有改进）。

&nbsp;
## 1.2 创建词汇表

- BPE 分词算法的目标是构建一个包含常见子词的词汇表，例如 `298: ent`（这个子词可以在 *entangle, entertain, enter, entrance, entity,* 等词中找到），甚至是完整的单词。


```
318: is
617: some
1212: This
2420: text
```

- BPE 算法最早在 1994 年由 Philip Gage 提出，详见：“[一种新的数据压缩算法](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)”。
- 在我们进入实际代码实现之前，今天用于 LLM 分词器的形式可以总结如下：

&nbsp;
## 1.3 BPE算法大览

**1. 识别出现次数最多的字节对**
- 在每次迭代中，扫描文本，找到最常出现的字节对（或字符对）。

**2. 替换并记录**

- 用一个新的占位符 ID 替换该字节对（该 ID 尚未被使用，例如，如果我们从 0...255 开始，第一个占位符将是 256）。
- 将该映射记录在查找表中。
- 查找表的大小是一个超参数，称为“词汇表大小”（对于 GPT-2，这个值为 50,257）。

**3. 重复直到没有进一步优化**

- 持续执行步骤 1 和 2，反复合并最频繁的字节对。
- 当无法再进行压缩时停止（例如，没有字节对出现超过一次）。

**解压缩（解码）**

- 为恢复原始文本，通过查找表将每个 ID 替换为其对应的字节对，逆向执行此过程。




&nbsp;
## 1.4 举个例子
### 1.4.1 Concrete example of the encoding part (steps 1 & 2)

- 假设训练数据集`the cat in the hat`，我们希望为 BPE 分词器构建词汇表。

**第一次迭代**

1. 识别最多出现字节对
  - 在这段文本中，"th" 出现了两次（分别位于开头和第二个 "e" 前）。

2. 替换并记录
  - 将 "th" 替换为一个尚未使用的新的令牌 ID，例如 256。
  - 新的文本变为：`<256>e cat in <256>e hat`。
  - 新的词汇表如下：


```
  0: ...
  ...
  256: "th"
```
**第二次迭代**

1. **识别最多出现字节对**  
   - 在文本 `<256>e cat in <256>e hat` 中，字节对 `<256>e` 出现了两次。

2. **替换并记录**  
   - 将 `<256>e` 替换为一个尚未使用的新的token ID，例如 257。  
   - 更新后的文本为：`<257> cat in <257> hat`。

     ```
     <257> cat in <257> hat
     ```
   - 更新后的将会是
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     ```
**第三次迭代**

1. **识别最多出现字节对**  
   - 在文本 `<257> cat in <257> hat` 中，字节对 `<257> ` 出现了两次（一次在开头，一次在“hat”之前）。

2. **替换并记录**  
   - 将 `<257> ` 替换为一个尚未使用的新的token ID，例如 258。  
   - 更新后的文本为：`<258>cat in <258>hat`。

     ```
     <258>cat in <258>hat
     ```
   - The updated vocabulary is:
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     258: "<257> "
     ```
     
- 如此循环

### 1.4.2 解码部分的具体示例（步骤 3）

- 为了恢复原始文本，我们通过逆向执行过程，将每个token ID 替换为它对应的字节对，按它们引入的逆序进行替换。
- 从最终压缩后的文本开始：`<258>cat in <258>hat`
- 替换 `<258>` → `<257> `：`<257> cat in <257> hat`
- 替换 `<257>` → `<256>e`：`<256>e cat in <256>e hat`
- 替换 `<256>` → "th"：`the cat in the hat`


&nbsp;
## 2. BPE的实现

- 以下是上述算法的具体代码，作为一个 Python 类，其模仿了 `tiktoken` Python 接口。
- 注意，上面的编码部分描述了通过 `train()` 进行的原始训练步骤；然而，`encode()` 方法的工作原理相似：

1. 将输入文本拆分为单个字节
2. 反复查找并替换（合并）相邻的令牌（字节对），当它们与学习到的 BPE 合并中的任何对匹配时（从最高到最低的“排名”，即按它们被学习的顺序）
3. 继续合并，直到不能再应用任何合并
4. 最终的token ID 列表就是编码后的输出

原始中文翻译的:

In [4]:
from collections import Counter, deque
from functools import lru_cache
import json


class BPETokenizerSimple:
    def __init__(self):
        # 映射 token_id 到 token_str（例如：{11246: "some"}）
        self.vocab = {}
        # 映射 token_str 到 token_id（例如：{"some": 11246}）
        self.inverse_vocab = {}
        # BPE 合并字典：{(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}
        # For the official OpenAI GPT-2 merges, use a rank dict:
        #  of form {(string_A, string_B): rank}, where lower rank = higher priority
        self.bpe_ranks = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        从头开始训练 BPE 分词器。

        参数：
            text (str): 训练文本。
            vocab_size (int): 目标词汇表大小。
            allowed_special (set): 要包含的特殊令牌集。
        """

        # 预处理：将空格替换为 'Ġ'
        # 注意，Ġ 是 GPT-2 BPE 实现的一个特性
        # 例如，"Hello world" 可能被标记为 ["Hello", "Ġworld"]
        # （GPT-4 BPE 会将其标记为 ["Hello", " world"]）
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # 初始化词汇表，包含唯一字符，包括 'Ġ'（如果存在）
        # 从前 256 个 ASCII 字符开始
        unique_chars = [chr(i) for i in range(256)]

        # 扩展 unique_chars，包含处理后的文本中未包含的字符
        unique_chars.extend(char for char in sorted(set(processed_text)) if char not in unique_chars)

        # 可选：确保 'Ġ' 包含在内（如果它对文本处理相关）
        if 'Ġ' not in unique_chars:
            unique_chars.append('Ġ')

        # 创建词汇表和逆词汇表
        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # 添加允许的特殊令牌
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # 将处理后的文本标记化为令牌 ID
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE 步骤 1-3：反复查找并替换频繁的字节对
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:  # 无更多字节对可合并，停止训练
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # 使用合并后的令牌构建词汇表
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id

    def load_vocab_and_merges_from_openai(self, vocab_path, bpe_merges_path):
        """
        从 OpenAI 的 GPT-2 文件加载预训练的词汇表和 BPE 合并。

        参数：
            vocab_path (str): 词汇文件路径（GPT-2 称其为 'encoder.json'）。
            bpe_merges_path (str): BPE 合并文件路径（GPT-2 称其为 'vocab.bpe'）。
        """
        # 加载词汇表
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            # loaded_vocab 将 token_str 映射到 token_id
            self.vocab = {int(v): k for k, v in loaded_vocab.items()}  # token_id: token_str
            self.inverse_vocab = {k: int(v) for k, v in loaded_vocab.items()}  # token_str: token_id

        # Handle newline character without adding a new token
        if "\n" not in self.inverse_vocab:
            # Use an existing token ID as a placeholder for '\n'
            # Preferentially use "<|endoftext|>" if available
            fallback_token = next((token for token in ["<|endoftext|>", "Ġ", ""] if token in self.inverse_vocab), None)
            if fallback_token is not None:
                newline_token_id = self.inverse_vocab[fallback_token]
            else:
                # If no fallback token is available, raise an error
                raise KeyError("No suitable token found in vocabulary to map '\\n'.")

            self.inverse_vocab["\n"] = newline_token_id
            self.vocab[newline_token_id] = "\n"

        # Load GPT-2 merges and store them with an assigned "rank"
        self.bpe_ranks = {}  # reset ranks

        # 加载 BPE 合并
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            # 如果有头部注释行，跳过它
            if lines and lines[0].startswith("#"):
                lines = lines[1:]
            rank = 0

            for line in enumerate(lines):
                pair = tuple(line.strip().split())
                if len(pair) != 2:
                    print(f"第 {rank+1} 行有超过 2 个条目：{line.strip()}")
                    continue
                token1, token2 = pair
                if token1 in self.inverse_vocab and token2 in self.inverse_vocab:
                    token_id1 = self.inverse_vocab[token1]
                    token_id2 = self.inverse_vocab[token2]
                    merged_token = token1 + token2
                    if merged_token in self.inverse_vocab:
                        merged_token_id = self.inverse_vocab[merged_token]
                        self.bpe_merges[(token_id1, token_id2)] = merged_token_id
                    else:
                        print(f"合并的令牌 '{merged_token}' 未在词汇表中找到，跳过。")
                else:
                    print(f"跳过配对 {pair}，因为其中一个令牌不在词汇表中。")

    def encode(self, text):
        """
        将输入文本编码为令牌 ID 列表。

        参数：
            text (str): 要编码的文本。

        返回：
            List[int]: 令牌 ID 列表。
        """
        tokens = []
        # 将文本拆分为令牌，确保换行符 intact
        words = text.replace("\n", " \n ").split()  # 确保 '\n' 被视为独立的令牌

        for i, word in enumerate(words):
            if i > 0 and not word.startswith("\n"):
                tokens.append("Ġ" + word)  # 如果单词前有空格或换行符，添加 'Ġ'
            else:
                tokens.append(word)  # 处理第一个单词或独立的 '\n'

        token_ids = []
        for token in tokens:
            if token in self.inverse_vocab:
                # token 已经存在于词汇表中
                token_id = self.inverse_vocab[token]
                token_ids.append(token_id)
            else:
                # 尝试通过 BPE 处理子词分词
                sub_token_ids = self.tokenize_with_bpe(token)
                token_ids.extend(sub_token_ids)

        return token_ids

    def tokenize_with_bpe(self, token):
        """
        使用 BPE 合并对单个令牌进行分词。

        参数：
            token (str): 要分词的令牌。

        返回：
            List[int]: 应用 BPE 后的令牌 ID 列表。
        """
        # 将令牌拆分为单个字符（作为初始令牌 ID）
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"未在词汇表中找到的字符：{missing_chars}")

        can_merge = True
        while can_merge and len(token_ids) > 1:
            can_merge = False
            new_tokens = []
            i = 0
            while i < len(token_ids) - 1:
                pair = (token_ids[i], token_ids[i + 1])
                if pair in self.bpe_merges:
                    merged_token_id = self.bpe_merges[pair]
                    new_tokens.append(merged_token_id)
                    i += 2  # 跳过下一个令牌，因为它已被合并
                    can_merge = True
                else:
                    new_tokens.append(token_ids[i])
                    i += 1
            if i < len(token_ids):
                new_tokens.append(token_ids[i])
            token_ids = new_tokens

        return token_ids

    def decode(self, token_ids):
        """
        将令牌 ID 列表解码回字符串。

        参数：
            token_ids (List[int]): 要解码的令牌 ID 列表。

        返回：
            str: 解码后的字符串。
        """
        decoded_string = ""
        for token_id in token_ids:
            if token_id not in self.vocab:
                raise ValueError(f"未在词汇表中找到令牌 ID {token_id}。")
            token = self.vocab[token_id]
            if token.startswith("Ġ"):
                # 用空格替换 'Ġ'
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
        return decoded_string

    def save_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        将词汇表和 BPE 合并保存到 JSON 文件。

        参数：
            vocab_path (str): 保存词汇表的路径。
            bpe_merges_path (str): 保存 BPE 合并的路径。
        """
        # 保存词汇表
        with open(vocab_path, "w", encoding="utf-8") as file:
            json.dump({k: v for k, v in self.vocab.items()}, file, ensure_ascii=False, indent=2)

        # 保存 BPE 合并作为字典列表
        with open(bpe_merges_path, "w", encoding="utf-8") as file:
            merges_list = [{"pair": list(pair), "new_id": new_id}
                           for pair, new_id in self.bpe_merges.items()]
            json.dump(merges_list, file, ensure_ascii=False, indent=2)

    def load_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        从 JSON 文件加载词汇表和 BPE 合并。

        参数：
            vocab_path (str): 词汇表文件路径。
            bpe_merges_path (str): BPE 合并文件路径。
        """
        # 加载词汇表
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            self.vocab = {int(k): v for k, v in loaded_vocab.items()}
            self.inverse_vocab = {v: int(k) for k, v in loaded_vocab.items()}

        # 加载 BPE 合并
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            merges_list = json.load(file)
            for merge in merges_list:
                pair = tuple(merge['pair'])
                new_id = merge['new_id']
                self.bpe_merges[pair] = new_id

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("无效模式。选择 'most' 或 'least'。")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                dq.popleft()
            else:
                replaced.append(current)

        return replaced


新的来自原始作者的:

In [7]:
from collections import Counter, deque
from functools import lru_cache
import json


class BPETokenizerSimple:
    def __init__(self):
        # Maps token_id to token_str (e.g., {11246: "some"})
        self.vocab = {}
        # Maps token_str to token_id (e.g., {"some": 11246})
        self.inverse_vocab = {}
        # Dictionary of BPE merges: {(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}

        # For the official OpenAI GPT-2 merges, use a rank dict:
        #  of form {(string_A, string_B): rank}, where lower rank = higher priority
        self.bpe_ranks = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        Train the BPE tokenizer from scratch.

        Args:
            text (str): The training text.
            vocab_size (int): The desired vocabulary size.
            allowed_special (set): A set of special tokens to include.
        """

        # Preprocess: Replace spaces with "Ġ"
        # Note that Ġ is a particularity of the GPT-2 BPE implementation
        # E.g., "Hello world" might be tokenized as ["Hello", "Ġworld"]
        # (GPT-4 BPE would tokenize it as ["Hello", " world"])
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # Initialize vocab with unique characters, including "Ġ" if present
        # Start with the first 256 ASCII characters
        unique_chars = [chr(i) for i in range(256)]
        unique_chars.extend(
            char for char in sorted(set(processed_text))
            if char not in unique_chars
        )
        if "Ġ" not in unique_chars:
            unique_chars.append("Ġ")

        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # Add allowed special tokens
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # Tokenize the processed_text into token IDs
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE steps 1-3: Repeatedly find and replace frequent pairs
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # Build the vocabulary with merged tokens
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id

    def load_vocab_and_merges_from_openai(self, vocab_path, bpe_merges_path):
        """
        Load pre-trained vocabulary and BPE merges from OpenAI's GPT-2 files.

        Args:
            vocab_path (str): Path to the vocab file (GPT-2 calls it 'encoder.json').
            bpe_merges_path (str): Path to the bpe_merges file  (GPT-2 calls it 'vocab.bpe').
        """
        # Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            # Convert loaded vocabulary to correct format
            self.vocab = {int(v): k for k, v in loaded_vocab.items()}
            self.inverse_vocab = {k: int(v) for k, v in loaded_vocab.items()}

        # Handle newline character without adding a new token
        if "\n" not in self.inverse_vocab:
            # Use an existing token ID as a placeholder for '\n'
            # Preferentially use "<|endoftext|>" if available
            fallback_token = next((token for token in ["<|endoftext|>", "Ġ", ""] if token in self.inverse_vocab), None)
            if fallback_token is not None:
                newline_token_id = self.inverse_vocab[fallback_token]
            else:
                # If no fallback token is available, raise an error
                raise KeyError("No suitable token found in vocabulary to map '\\n'.")

            self.inverse_vocab["\n"] = newline_token_id
            self.vocab[newline_token_id] = "\n"

        # Load GPT-2 merges and store them with an assigned "rank"
        self.bpe_ranks = {}  # reset ranks
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            if lines and lines[0].startswith("#"):
                lines = lines[1:]

            rank = 0
            for line in lines:
                pair = tuple(line.strip().split())
                if len(pair) == 2:
                    token1, token2 = pair
                    # If token1 or token2 not in vocab, skip
                    if token1 in self.inverse_vocab and token2 in self.inverse_vocab:
                        self.bpe_ranks[(token1, token2)] = rank
                        rank += 1
                    else:
                        print(f"Skipping pair {pair} as one token is not in the vocabulary.")

    def encode(self, text, allowed_special=None):
        """
        Encode the input text into a list of token IDs, with tiktoken-style handling of special tokens.

        Args:
            text (str): The input text to encode.
            allowed_special (set or None): Special tokens to allow passthrough. If None, special handling is disabled.

        Returns:
            List of token IDs.
        """
        import re

        token_ids = []

        # If special token handling is enabled
        if allowed_special is not None and len(allowed_special) > 0:
            # Build regex to match allowed special tokens
            special_pattern = (
                "(" + "|".join(re.escape(tok) for tok in sorted(allowed_special, key=len, reverse=True)) + ")"
            )

            last_index = 0
            for match in re.finditer(special_pattern, text):
                prefix = text[last_index:match.start()]
                token_ids.extend(self.encode(prefix, allowed_special=None))  # Encode prefix without special handling

                special_token = match.group(0)
                if special_token in self.inverse_vocab:
                    token_ids.append(self.inverse_vocab[special_token])
                else:
                    raise ValueError(f"Special token {special_token} not found in vocabulary.")
                last_index = match.end()

            text = text[last_index:]  # Remaining part to process normally

            # Check if any disallowed special tokens are in the remainder
            disallowed = [
                tok for tok in self.inverse_vocab
                if tok.startswith("<|") and tok.endswith("|>") and tok in text and tok not in allowed_special
            ]
            if disallowed:
                raise ValueError(f"Disallowed special tokens encountered in text: {disallowed}")

        # If no special tokens, or remaining text after special token split:
        tokens = []
        lines = text.split("\n")
        for i, line in enumerate(lines):
            if i > 0:
                tokens.append("\n")
            words = line.split()
            for j, word in enumerate(words):
                if j == 0 and i > 0:
                    tokens.append("Ġ" + word)
                elif j == 0:
                    tokens.append(word)
                else:
                    tokens.append("Ġ" + word)

        for token in tokens:
            if token in self.inverse_vocab:
                token_ids.append(self.inverse_vocab[token])
            else:
                token_ids.extend(self.tokenize_with_bpe(token))

        return token_ids

    def tokenize_with_bpe(self, token):
        """
        Tokenize a single token using BPE merges.

        Args:
            token (str): The token to tokenize.

        Returns:
            List[int]: The list of token IDs after applying BPE.
        """
        # Tokenize the token into individual characters (as initial token IDs)
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        # If we haven't loaded OpenAI's GPT-2 merges, use my approach
        if not self.bpe_ranks:
            can_merge = True
            while can_merge and len(token_ids) > 1:
                can_merge = False
                new_tokens = []
                i = 0
                while i < len(token_ids) - 1:
                    pair = (token_ids[i], token_ids[i + 1])
                    if pair in self.bpe_merges:
                        merged_token_id = self.bpe_merges[pair]
                        new_tokens.append(merged_token_id)
                        # Uncomment for educational purposes:
                        # print(f"Merged pair {pair} -> {merged_token_id} ('{self.vocab[merged_token_id]}')")
                        i += 2  # Skip the next token as it's merged
                        can_merge = True
                    else:
                        new_tokens.append(token_ids[i])
                        i += 1
                if i < len(token_ids):
                    new_tokens.append(token_ids[i])
                token_ids = new_tokens
            return token_ids

        # Otherwise, do GPT-2-style merging with the ranks:
        # 1) Convert token_ids back to string "symbols" for each ID
        symbols = [self.vocab[id_num] for id_num in token_ids]

        # Repeatedly merge all occurrences of the lowest-rank pair
        while True:
            # Collect all adjacent pairs
            pairs = set(zip(symbols, symbols[1:]))
            if not pairs:
                break

            # Find the pair with the best (lowest) rank
            min_rank = float("inf")
            bigram = None
            for p in pairs:
                r = self.bpe_ranks.get(p, float("inf"))
                if r < min_rank:
                    min_rank = r
                    bigram = p

            # If no valid ranked pair is present, we're done
            if bigram is None or bigram not in self.bpe_ranks:
                break

            # Merge all occurrences of that pair
            first, second = bigram
            new_symbols = []
            i = 0
            while i < len(symbols):
                # If we see (first, second) at position i, merge them
                if i < len(symbols) - 1 and symbols[i] == first and symbols[i+1] == second:
                    new_symbols.append(first + second)  # merged symbol
                    i += 2
                else:
                    new_symbols.append(symbols[i])
                    i += 1
            symbols = new_symbols

            if len(symbols) == 1:
                break

        # Finally, convert merged symbols back to IDs
        merged_ids = [self.inverse_vocab[sym] for sym in symbols]
        return merged_ids

    def decode(self, token_ids):
        """
        Decode a list of token IDs back into a string.

        Args:
            token_ids (List[int]): The list of token IDs to decode.

        Returns:
            str: The decoded string.
        """
        decoded_string = ""
        for i, token_id in enumerate(token_ids):
            if token_id not in self.vocab:
                raise ValueError(f"Token ID {token_id} not found in vocab.")
            token = self.vocab[token_id]
            if token == "\n":
                if decoded_string and not decoded_string.endswith(" "):
                    decoded_string += " "  # Add space if not present before a newline
                decoded_string += token
            elif token.startswith("Ġ"):
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
        return decoded_string

    def save_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        Save the vocabulary and BPE merges to JSON files.

        Args:
            vocab_path (str): Path to save the vocabulary.
            bpe_merges_path (str): Path to save the BPE merges.
        """
        # Save vocabulary
        with open(vocab_path, "w", encoding="utf-8") as file:
            json.dump(self.vocab, file, ensure_ascii=False, indent=2)

        # Save BPE merges as a list of dictionaries
        with open(bpe_merges_path, "w", encoding="utf-8") as file:
            merges_list = [{"pair": list(pair), "new_id": new_id}
                           for pair, new_id in self.bpe_merges.items()]
            json.dump(merges_list, file, ensure_ascii=False, indent=2)

    def load_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        Load the vocabulary and BPE merges from JSON files.

        Args:
            vocab_path (str): Path to the vocabulary file.
            bpe_merges_path (str): Path to the BPE merges file.
        """
        # Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            self.vocab = {int(k): v for k, v in loaded_vocab.items()}
            self.inverse_vocab = {v: int(k) for k, v in loaded_vocab.items()}

        # Load BPE merges
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            merges_list = json.load(file)
            for merge in merges_list:
                pair = tuple(merge["pair"])
                new_id = merge["new_id"]
                self.bpe_merges[pair] = new_id

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if not pairs:
            return None

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'.")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # Remove the 2nd token of the pair, 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)

        return replaced

- 上面 `BPETokenizerSimple` 类的代码比较多，详细的内容超出了本笔记本的范围，但下一节提供了一个简短的使用概述，用来帮助您更好地理解该类的方法。


## 3. BPE演示

- 实际上，我强烈推荐使用 [tiktoken](https://github.com/openai/tiktoken)，因为我上面的实现更注重可读性和教育目的，无法保证性能。
- 然而，使用方式与 tiktoken 基本相似，不同之处在于 tiktoken 没有训练方法。
- 让我们通过以下几个示例来看看我上面的 `BPETokenizerSimple` Python 代码是如何工作的。


### 3.1 训练、编码与解码

中文翻译版本的:

In [6]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    text = f.read()

原始作者的:

In [ ]:
import os
import urllib.request

def download_file_if_absent(url, filename, search_dirs):
    for directory in search_dirs:
        file_path = os.path.join(directory, filename)
        if os.path.exists(file_path):
            print(f"{filename} already exists in {file_path}")
            return file_path

    target_path = os.path.join(search_dirs[0], filename)
    try:
        with urllib.request.urlopen(url) as response, open(target_path, "wb") as out_file:
            out_file.write(response.read())
        print(f"Downloaded {filename} to {target_path}")
    except Exception as e:
        print(f"Failed to download {filename}. Error: {e}")
    return target_path

verdict_path = download_file_if_absent(
    url=(
         "https://raw.githubusercontent.com/rasbt/"
         "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
         "the-verdict.txt"
    ),
    filename="the-verdict.txt",
    search_dirs=["ch02/01_main-chapter-code/", "../01_main-chapter-code/", "."]
)

with open(verdict_path, "r", encoding="utf-8") as f: # added ../01_main-chapter-code/
    text = f.read()

- 接下来，我们将训练一个 BPE 分词器，词汇表大小为 1,000。
- 词汇表大小默认已经为 255，因此我们实际上只“学习”了 745 个词汇条目。
- 作为对比，GPT-2 的词汇表包含 50,257 个token，GPT-4 的词汇表包含 100,256 个令牌（在 tiktoken 中为 `cl100k_base`），GPT-4o 使用 199,997 个token（在 tiktoken 中为 `o200k_base`）；与我们简单的示例文本相比，它们拥有更大的训练集。


In [8]:
tokenizer = BPETokenizerSimple()
tokenizer.train(text, vocab_size=1000, allowed_special={"<|endoftext|>"})

- 查看下vocab里的东西

In [9]:
# print(tokenizer.vocab)
print(len(tokenizer.vocab))

1000


- 合并了 742 次 (~ `1000 - len(range(0, 256))`)
- 742 times (= 1000 - len(range(0, 256)) - len(special_tokens) - "Ġ" = 1000 - 256 - 1 - 1 = 742)

In [10]:
print(len(tokenizer.bpe_merges))

742


- 这代表前256是单token单次

- 接下来我们要`编码`点内容

In [15]:
input_text = "Jack embraced beauty through art and life."
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [16]:
print("Number of characters:", len(input_text))
print("Number of token IDs:", len(token_ids))

Number of characters: 42
Number of token IDs: 20


In [12]:
input_text = "Jack embraced beauty through art and life.<|endoftext|> "
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 60, 124, 271, 683, 102, 116, 461, 116, 124, 62]


In [13]:
input_text = "Jack embraced beauty through art and life.<|endoftext|> "
token_ids = tokenizer.encode(input_text, allowed_special={"<|endoftext|>"})
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 257]


- 从上述长度可以看出，一个 42 字符的句子被编码为 20 个令牌 ID，相比于基于字符字节的编码，这有效地将输入长度缩短了大约一半。

- 请注意，词汇表本身在 `decoder()` 方法中被使用，该方法允许我们将令牌 ID 映射回文本：

In [17]:
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [18]:
print(tokenizer.decode(token_ids))

Jack embraced beauty through art and life.


- 通过遍历每个令牌 ID，我们可以更好地理解令牌 ID 是如何通过词汇表解码的：

In [19]:
for token_id in token_ids:
    print(f"{token_id} -> {tokenizer.decode([token_id])}")

424 -> Jack
256 ->  
654 -> em
531 -> br
302 -> ac
311 -> ed
256 ->  
296 -> be
97 -> a
465 -> ut
121 -> y
595 ->  through
841 ->  ar
116 -> t
287 ->  a
466 -> nd
256 ->  
326 -> li
972 -> fe
46 -> .


- 如我们所见，大多数token ID 表示 2 字符的子词；这是因为训练数据文本非常简短，重复的单词并不多，而且我们使用了相对较小的词汇表大小。

- 总结来说，调用 `decode(encode())` 应该能够重现任意输入文本：

In [20]:
tokenizer.decode(tokenizer.encode("This is some text."))

'This is some text.'

In [21]:
tokenizer.decode(
    tokenizer.encode("This is some text with \n newline characters.")
)

'This is some text with \n newline characters.'

### 3.2 储存tokenizer

- 接下来，让我们看看如何保存训练好的分词器，以便稍后重新使用：

In [22]:
# Save trained tokenizer
tokenizer.save_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

In [23]:
# Load tokenizer
tokenizer2 = BPETokenizerSimple()
tokenizer2.load_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

- 加载的分词器应该能够产生与之前相同的结果。

In [24]:
print(tokenizer2.decode(token_ids))

Jack embraced beauty through art and life.


In [25]:
tokenizer2.decode(
    tokenizer2.encode("This is some text with \n newline characters.")
)

'This is some text with \n newline characters.'

&nbsp;
### 3.3 加载来自 OpenAI 的原始 GPT-2 BPE 分词器

- 最后，让我们加载 OpenAI 的 GPT-2 分词器文件。

中文翻译的:

In [26]:
import os
import urllib.request

def download_file_if_absent(url, filename):
    if not os.path.exists(filename):
        try:
            with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
                out_file.write(response.read())
            print(f"Downloaded {filename}")
        except Exception as e:
            print(f"Failed to download {filename}. Error: {e}")
    else:
        print(f"{filename} already exists")

files_to_download = {
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe": "vocab.bpe",
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json": "encoder.json"
}

for url, filename in files_to_download.items():
    download_file_if_absent(url, filename)

Downloaded vocab.bpe
Downloaded encoder.json


原始作者的:

In [ ]:
# Download files if not already present in this directory

# Define the directories to search and the files to download
search_directories = ["ch02/02_bonus_bytepair-encoder/gpt2_model/", "../02_bonus_bytepair-encoder/gpt2_model/", "."]

files_to_download = {
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe": "vocab.bpe",
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json": "encoder.json"
}

# Ensure directories exist and download files if needed
paths = {}
for url, filename in files_to_download.items():
    paths[filename] = download_file_if_absent(url, filename, search_directories)

- 接下来，我们通过 `load_vocab_and_merges_from_openai` 方法加载文件：

In [27]:
tokenizer_gpt2 = BPETokenizerSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path="encoder.json", bpe_merges_path="vocab.bpe"
)

- 词汇表大小应为 `50257`，我们可以通过下面的代码进行确认：

In [28]:
len(tokenizer_gpt2.vocab)

50257

- 现在我们可以通过我们的 `BPETokenizerSimple` 对象使用 GPT-2 分词器：

In [29]:
input_text = "This is some text"
token_ids = tokenizer_gpt2.encode(input_text)
print(token_ids)

[1212, 318, 617, 2420]


In [30]:
print(tokenizer_gpt2.decode(token_ids))

This is some text


In [31]:
import tiktoken

tik_tokenizer = tiktoken.get_encoding("gpt2")
tik_tokenizer.encode(input_text)

[1212, 318, 617, 2420]

- 你可以通过互动式的 [tiktoken 应用](https://tiktokenizer.vercel.app/?model=gpt2) 或 [tiktoken 库](https://github.com/openai/tiktoken)来确认这是否生成了正确的token：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="600px">


```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints [1212, 318, 617, 2420]
```


&nbsp;
# 4. 结论

- 这就是 BPE 的基本原理，包括用于创建新分词器的训练方法，或者从原始 OpenAI GPT-2 模型加载 GPT-2 分词器的词汇表和合并。
- 希望这个简短的教程对教学目的有所帮助；如果你有任何问题，请随时在 [这里](https://github.com/rasbt/LLMs-from-scratch/discussions/categories/q-a) 开启一个新的讨论。
- 关于与其他分词器实现的性能对比，请参见 [这个笔记本](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb)。


BPE（Byte Pair Encoding）是一种用于**子词分词（subword tokenization）**的算法，广泛应用于自然语言处理任务中，尤其是在像 GPT-2 这样的语言模型中。下面我来详细解释它的基本原理、训练方法，以及如何从 GPT-2 模型加载词汇表和合并规则。

---

## 🧠 BPE 的基本原理

BPE 最初是一种数据压缩算法，后来被用于 NLP 中的分词任务。它的核心思想是：

> **通过频率合并字符或子词对，逐步构建一个子词词汇表。**

### 步骤如下：

1. **初始化：**
   - 将训练语料中的每个单词拆分为字符序列。例如，`low` → `l o w`。
   - 在每个单词末尾加上特殊符号（如 `</w>`）表示词尾。

2. **统计频率：**
   - 统计所有相邻字符对（或子词对）的出现频率。

3. **合并频率最高的对：**
   - 找出频率最高的字符对（如 `l o`），将其合并为一个新符号（如 `lo`）。

4. **重复步骤 2 和 3：**
   - 直到达到预设的合并次数（即词汇表大小）或没有更多可合并的对。

最终结果是一个包含常见子词的词汇表，可以有效处理未登录词（OOV）问题。

---

## 🛠️ 用于创建新分词器的训练方法

如果你想自己训练一个 BPE 分词器，可以使用以下工具：

### 常用工具：
- **SentencePiece**
- **tokenizers** by Hugging Face
- **subword-nmt**

### 基本流程：

1. 准备语料文件（纯文本）。
2. 使用工具训练 BPE 模型：
   - 例如使用 `subword-nmt`：
     ```bash
     subword-nmt learn-bpe -s 10000 < corpus.txt > bpe.codes
     subword-nmt apply-bpe -c bpe.codes < corpus.txt > corpus.bpe
     ```
3. 得到：
   - `bpe.codes`：合并规则。
   - `vocab.txt`：词汇表。

---

## 📦 从 GPT-2 模型加载词汇表和合并规则

GPT-2 使用的是 BPE 分词器，词汇表和合并规则可以从 OpenAI 的开源代码中获取：

### 文件位置（在 Hugging Face Transformers 中）：
- `vocab.json`：词汇表（token 到 id 的映射）
- `merges.txt`：BPE 合并规则（每一行是一个合并对）

### 加载方式（使用 Hugging Face）：

```python
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# 查看词汇表和合并规则
print(tokenizer.vocab)         # 或 tokenizer.get_vocab()
print(tokenizer.bpe_merges)    # 合并规则
```

你也可以自定义加载：

```python
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=10000, show_progress=True)
tokenizer.train(["corpus.txt"], trainer)
```